# 3 More evolutionary and advanced considerations


## Multiple Scores
## Population of results/estimates
## Population diversity
## On the fly learning


The first chunk of code is boilerplate, as usual, just more can be considered boilerplate now.

In [ ]:
#boilerplate python and reading in data
import sys
import csv
from math import *
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Particular to the evolution
from scores import *
from evolution3 import *

# Some global parameters:
nobs = 579
nparameters = 6

npopulation = 10
per_second = 60     # estimate of number of generations per second
genmax = int(60*per_second)

train_start = int(0)
train_end   = int(364)
np.random.seed(0)      # for reproducibility

######################## ######################## ########################
# Now bring in the data for real work:
matchup_set = []

with open('testin.csv') as csvfile:
    k = 0
    sreader = csv.reader(csvfile, delimiter=",")
    for line in sreader:
        day     = float(line[0])
        t2m_gfs = float(line[1])
        td_gfs  = float(line[2])
        thick_gfs = float(line[3])
        rh_gfs  = float(line[4])
        speed   = float(line[5])
        obs_t2m = float(line[6])
        obs_td  = float(line[7])
        terr    = float(line[8])
        tderr  = float(line[9])

        #Note that obs_td, obs_t2m, tderr are being ignored. They can be
        #       added to the list.
        #  n.b.: note that it is terr that is used, not t2m itself.
        #Model and observation are well-enough correlated that it is the increment
        #which makes more sense to predict [Krasnopolsky,20NNN]
        m = matchup((day,t2m_gfs,td_gfs,thick_gfs,rh_gfs,speed,terr))
        matchup_set.append(m)
        k += 1

csvfile.close()
######################## ######################## ########################

#Initialize and seed the population, best, and good
population = []
bests      = []       # Save all then-best versions in memory
goods      = []       # Save everything better than raw GFS to file (could be a lot)
fgood = open("goods","w")

for k in range (0,npopulation):
    population.append(critter(nparameters))

weights = np.zeros((nparameters))
sdevs   = np.zeros((nparameters))

bests.append(critter(nparameters))
bests[0].init(weights, sdevs, 99.)
nbests = 1

goods.append(critter(nparameters))
goods[0].init(weights, sdevs, 99.)
ngoods = 1



If you go to the 'home' screen, you'll see that the file 'goods' now exists.

Now ready to start looking in to our more advanced topics, starting with

## 2 Multiple scores

In [ ]:
#Change this and re-run to see the effects of changes to the metric used:
#  Options are:
#  RMS (default)
#  MEAN
#  RM3 (root mean cubed error, will be related to skew)
#  RM4 (root mean quartic error, will be related to kurtosis)
#  MAE (Mean absolute error)
#  NLOSS (count the number of times the corrected output is worse than the original); 
#      not currently working, will fall back to RMS
#  VICKIE (MAE with a 3 C tolerance -- errors less than 3 C are ignored, otherwise, MAE)
#    -- a representation of a real person's (my wife) interest in T2M accuracy.
#The 'scores' file imported above contains the code and it is easy to add or modify

#We could also make this a loop itself, and would outside of a notebook. 
#  But, in notebook, let's take advantage of the interactive nature.
#  Try also, at least, MEAN and MAE
measure = RMS

#for reference, take the raw gfs output's score:
population[0].init(weights, sdevs, 99.)
score_gfs = population[0].skill(matchup_set, train_start, train_end, metric = measure)

print("uncorrected score in training period: ",
         population[0].skill(matchup_set, train_start, train_end, metric = measure) )
print("uncorrected score in evaluation period: ",
         population[0].skill(matchup_set, train_end+1, nobs, metric = measure), flush=True )
population[0].show_fcst(matchup_set, train_start, train_end)
print("\n",flush=True)

#For bias + linear terms
for k in range (0,int(6)):
    sdevs[k] = 1.0

#Initialize the population
for k in range (0,npopulation):
  for l in range (0, int(6) ):  
      weights[l] = np.random.normal(0,sdevs[l])
  population[k].init(weights,sdevs, 99.)

#Find our first 'best' -- noting that we aren't using raw gfs as a member of the population
smin = 9999.
kbest = int(npopulation)
for k in range (0,npopulation):
    population[k].skill(matchup_set, train_start, train_end, metric = measure)
    if (population[k].score < smin):
        kbest = k
        smin = population[k].score

#Start accumulating our best critters
bests.append(critter(nparameters))
bests[nbests].init(population[kbest].weights, population[kbest].sdevs, population[kbest].score)
nbests += 1

# ... and ones better than raw gfs
if (smin < score_gfs):
    goods.append(critter(nparameters))
    goods[ngoods].init(population[kbest].weights, population[kbest].sdevs, population[kbest].score)
    ngoods += 1

population[kbest].show()
print("initial kbest, smin = ",kbest, smin, flush=True)

As before, now run through the generations. But this time, save all the 'good' parameter settings, not just the best. We'll examine them for ideas.

In [ ]:
for gen in range(0,genmax):
    if ((gen%per_second) == 0):
        print("generation ", gen, flush=True)

    population[0].copy(population[kbest])
    population[0].score = population[kbest].score
    score_best = float(population[0].score)
    smin = score_best
    kbest = 0
    for k in range (1, npopulation):
        population[k].copy(population[0])
        population[k].evolve()
        population[k].skill(matchup_set, train_start, train_end, metric = RMS)
        if (population[k].score < score_best):
            kbest = k
            smin = population[k].score
            bests.append(critter(nparameters))
            bests[nbests].init(population[kbest].weights, population[kbest].sdevs, population[kbest].score)
            nbests += 1
        #Save the ones that are better than the raw gfs
        if (population[k].score < score_gfs):
            kgood = k
            goods.append(critter(nparameters))
            goods[ngoods].score = population[k].score
            goods[ngoods].init(population[kgood].weights, population[kgood].sdevs, population[k].score)
            population[k].show(fgood)  #Print this 'good' result to file for safe-keeping
            ngoods += 1

    if (kbest != 0):
        if (score_gfs != 0):
          print("new best ",gen, kbest, smin, score_best, smin/score_gfs, flush=True)
        else:
          print("new best ",gen, kbest, smin, score_best, flush=True)
        population[kbest].show()
        
print("done with generations")

In [ ]:
######################## ######################## ########################
if (score_gfs != 0):
  print("best score in training period ",gen, kbest, smin, score_best, smin/score_gfs, flush=True)
else:
  print("best score in training period ",gen, kbest, smin, score_best, flush=True)
print("score in the untrained period: ",population[kbest].skill(matchup_set, train_end+1, nobs, metric = measure))

print("found ",ngoods," good solutions")

for k in range (0, nbests):
  print("best k ",k,bests[k].score)
  bests[k].show()
  print("\n")



In [ ]:
print("Forecasts in the training period:")
population[0].show_fcst(matchup_set, train_start, train_end)

In [ ]:
print("Untrained forecasts:")
population[0].show_fcst(matchup_set, train_end, nobs)
population[0].skill(matchup_set, train_end+1, nobs, metric = measure)

## Considering different scores:

(warning, I may have failed to properly re-initialize metrics, this is place holder text, not finished)

### After optimizing for RMS:
trained period has 44% the RMS of raw GFS (16.79)

trained mean rms  -0.1532608638628728 7.452270353231618  (44%)

raw gfs training period mean rms  13.939368131868132 16.79408375517474

untrained mean rms  -1.6085981736055173 5.385852764158143

raw gfs untrained period: 11.899786904482491 


### Optimizing for MAE
uncorrected score in training period:  14.072794520547944

uncorrected score in evaluation period:  10.25293023255814

best score in training period  3599 0 7.451589648287305 7.451589648287305 0.5295031940818224

trained mean rms  2.3770208862705613 9.513703004818282

score in the untrained period:  4.331689063263931

untrained mean rms  0.946926616881068 5.606120899560049

### Optimizing for Vickie
uncorrected score in training period:  15.619538461538465

uncorrected score in evaluation period:  11.043857868020302

trained mean rms  -0.1532608638628728 7.452270353231618 #### note same as for RMS

best score in training period  3599 0 7.442054761927269 7.442054761927269 0.4764580451754433

untrained mean rms  -1.6085981736055173 5.385852764158143

untrained score in evaluation period: 6.169421042958432


### Optimizing for Mean
uncorrected score in training period:  13.901178082191784

uncorrected score in evaluation period:  10.209953488372092

best score in training period  3599 0 2.3705085002807773 2.3705085002807773 0.17052572711931058

score in the untrained period:  0.9707819312466325


### Optimizing for RM4

uncorrected score in training period:  20.17958780895671

uncorrected score in evaluation period:  14.35790534535554

best score in training period  3599 0 7.442054761927269 7.442054761927269 0.3687912177583782

trained mean rms  -0.1532608638628728 7.452270353231618

untrained mean rms  -1.6085981736055173 5.385852764158143

score in the untrained period:  7.567201668271535

## Looking in to the nature of the 'good' results

We now have a large number (typically a few thousand, for a population of 10 per generation and 3600 generations) of parameter sets / regression coefficients, and the (training period) score for them. In the case of a typical regression, we would be done here.

As an evolutionary matter, we have more that we may examine.


In [ ]:
# Extract the weights and the skill in the training set that corresponded to good results
#print(ngoods, nparameters)

consts = np.zeros((ngoods, nparameters))
skill = np.zeros((ngoods))

for i in range(0,ngoods):
# Both this and the commented-out segment work, but this is more pythonic
    consts[i] = goods[i].weights
#    for k in range (0,nparameters):
#        consts[i,k] = goods[i].weights[k]
    skill[i]  = goods[i].score

print(skill.min(), skill.mean(), len(skill))

In [ ]:
# Take a look at the coefficients for T2m and Td in the regression
fig,ax = plt.subplots()
ax.set(xlabel="GFS T2m coefficient", ylabel="GFS Td coefficient")
ax.scatter(consts[:,1],consts[:,2])
ax.grid()
plt.show()

In [ ]:
# Masked array support, so we can work with selected parts of the arrays/vectors
import numpy.ma as ma

good1 = np.zeros((ngoods))
good2 = np.zeros((ngoods))

# Since we know the minimum and mean scores, try plotting only those points for which the 
#    score is better than the mean
mask1 = ma.masked_array(skill < (skill.mean() + skill.min())/2. )
tcount = 0
for i in range(0,ngoods):
    if (mask1[i]):
        #print(skill[i])
        good1[i] = consts[i,1]
        good2[i] = consts[i,2]
        tcount += 1
        
print("found ",tcount," points")
# Take a look at the coefficients for T2m and Td in the regression
fig,ax = plt.subplots()
ax.set(xlabel="GFS T2m coefficient", ylabel="GFS Td coefficient")
ax.scatter(good1, good2)
ax.grid()
plt.show()    


Now try overlaying the better coefficients against all that were 'good'

In [ ]:
fig, ax = plt.subplots()
ax.set(xlabel="GFS T2m coefficient", ylabel="GFS Td coefficient")
ax.scatter(consts[:,1],consts[:,2], color = "black")
ax.scatter(good1, good2, color = "blue")
ax.grid()
plt.show()

Try for the 'very good' coefficients, those within 10% of the best (vs best-gfs)


In [ ]:
vgood1 = np.zeros((ngoods))
vgood2 = np.zeros((ngoods))

# Since we know the minimum and mean scores, try plotting only those points for which the 
#    score is better than the mean
mask2 = ma.masked_array(skill < (skill.min() + (score_gfs - skill.min())*0.1 ))
tcount2 = 0
for i in range(0,ngoods):
    if (mask2[i]):
        #print(skill[i])
        vgood1[i] = consts[i,1]
        vgood2[i] = consts[i,2]
        tcount2 += 1
        
print("found ",tcount2," points")
print(vgood2.min())
                        
fig, ax = plt.subplots()
ax.set(xlabel="GFS T2m coefficient", ylabel="GFS Td coefficient")
ax.scatter(consts[:,1],consts[:,2], color = "black")
ax.scatter(good1, good2, color = "blue")
ax.scatter(vgood1, vgood2, color = "red")
ax.grid()
plt.show()

### Considerations:

We see a definite progression, in that the better (lower) a score we require, the tighter a cloud that is found. On the other hand, the GFS T2m coefficient has quite a large allowable range -- from -1 to +1, while the GFS Td coefficient has an even larger range -- 0 to -3. Optically, at least, there's a very strong suggestion that the thing which is important is not T2m or Td, but, the difference between the two. Maybe a linear function of the difference, since the slope looks more like 1.5 than 1.

Is there a geophysical process or phenomenon that would depend on the difference between T2m and Td? The one that occurs to me is radiation from the surface layer. A drier atmosphere will be different than a wetter one regarding outgoing longwave, or, perhaps (via low level clouds) shortwave.

Simple-minded use of multiple regression could miss this. Our rather simple consideration from an evolutionary method catches this relation that can be used for further model diagnostics -- not just the correction of model output.

In the evolutionary method, we could now make (GFS T2m-Td) one of the variables used, directly, in the prediction, and its coefficient would be one of those we ask evolution to discover for us. Further, if we had reason (say because we, the some of we who are not me, understand radiative transfer well enough) to take a particular nonlinear function of T2m and Td, we could ask the evolutionary method to find _that_ function's parameters. 

Most simply, we could add a 7th parameter to our weights and standard deviations set, and make that the coefficient of (GFS T2m-td) in the predictor. Absolutely no meaningful change to the main program (nparameters has to be changed), just in the predictor code.